# Notebook for data ingestion

This notebook can be used for transforming markdown generated from [LLamaParse](https://docs.llamaindex.ai/en/stable/llama_cloud/llama_parse/) to a hierarchical graph database. The markdown files hierarchy needs to be set by marking the section headings with hashtags. Due to LLamaParse currently only outputting single hashtags some manual additions to the hashtag count is required to represent the correct hierarchy depth. 

The data ingestion process can be summarized as follows:

![Data ingestion process](./img/data_ingestion.png)

This notebook represents the second arrow named *Python-Algorithmus*, transforming the adjusted markdown to a graph structure. After creating the graph structure, the embeddings can be generated by executing the `voyage_embed.ipynb` notebook.

The database schema of the resulting graph is visualized below:

![Data ingestion process](./img/db_schema.png)

Steps:
- parse the PDF document of the standard to be transformed using **LLamaParse**
- create a neo4j instance and insert your credentials (`neo4j_uri`, `username`, `password`) into the corresponding cell
- make sure that the `neo4j` package is installed
- run the cells below in order

In [5]:
# class and function definitions for the markdown parser
import re

class Section:
    def __init__(self, name, level):
        self.name = name
        self.level = level
        self.content = []
        self.subsections = []

    def add_content(self, content):
        self.content.append(content)

    def add_subsection(self, subsection):
        self.subsections.append(subsection)

    def __repr__(self):
        return f"Section(name='{self.name}', level={self.level}, content={self.content}, subsections={self.subsections})"

def parse_markdown(markdown_text):
    lines = markdown_text.split('\n')
    root = Section("Root", 0)
    stack = [root]

    for line in lines:
        if line.startswith('#'):
            match = re.match(r'^(#+)\s*(.+)$', line)
            if match:
                level = len(match.group(1))
                name = match.group(2).strip()

                while level <= stack[-1].level:
                    stack.pop()

                new_section = Section(name, level)
                stack[-1].add_subsection(new_section)
                stack.append(new_section)
        elif line.strip():
            if stack[-1].content and stack[-1].content[-1]:
                stack[-1].content[-1] += '\n' + line
            else:
                stack[-1].add_content(line)
        else:
            if stack[-1].content:
                stack[-1].add_content('')

    return root

#def print_structure(section, indent=''):
#    print(f"{indent}{section.name} (Level {section.level})")
#    for content in section.content:
#        if content:
#            print(f"{indent}  Content: {content}")
#    for subsection in section.subsections:
#        print_structure(subsection, indent + '  ')

def print_structure(section, indent=''):
    print(f"{indent}{section.name} (Level {section.level})")
    for content in section.content:
        if content:
            # Replace linebreaks with spaces
            cleaned_content = content.replace('\n', ' ')
            print(f"{indent}  Content: {cleaned_content}")
    for subsection in section.subsections:
        print_structure(subsection, indent + '  ')

In [12]:
# paste the markdown output from LLamaParse here

markdown_text="""

# NA.1 Anwendungsbereich

Dieser Nationale Anhang enthält nationale Festlegungen für die Grundsätze zur Bestimmung der Werte von Schneelasten für die Berechnung und Bemessung von Hoch- und Ingenieurbauten, die bei der Anwendung von DIN EN 1991-1-3 und DIN EN 1991-1-3/A1 in Deutschland zu berücksichtigen sind.

Dieser Nationale Anhang gilt nur in Verbindung mit DIN EN 1991-1-3 und DIN EN 1991-1-3/A1.

# NA.2 Nationale Festlegungen zur Anwendung von DIN EN 1991-1-3 und DIN EN 1991-1-3/A1

## NA 2.1 Allgemeines

DIN EN 1991-1-3 weist an den folgenden Textstellen die Möglichkeit nationaler Festlegungen (NDP) aus:

- 1.1(2), 1.1.(3), 1.1.(4)
- 2(3), 2(4)
- 3.3(1), 3.3(2), 3.3(3)
- 4.1(1), 4.1(2), 4.2(1), 4.3(1)
- 5.2(2), 5.2(5), 5.2(6), 5.2(7), 5.2(8), 5.3.1(1) Anmerkung zu Tabelle 5.2, 5.3.2(3), 5.3.3(4), 5.3.4(3), 5.3.4(4), 5.3.5(1), 5.3.5(3), 5.3.6(1), 5.3.6(3)
- 6.2(2), 6.3(1), 6.3(2)
- A(1) (in Tabelle A.1)

Darüber hinaus enthält NA 2.2 ergänzende nicht widersprechende Angaben zur Anwendung von DIN EN 1991-1-3 und DIN EN 1991-1-3/A1. Diese sind durch ein vorangestelltes „NCI“ gekennzeichnet.

- 5.3.1(2)
- 6.4(1)
- Anhang NA.F.

## NA 2.2 Nationale Festlegungen

ANMERKUNG Die nachfolgende Nummerierung und die Überschriften entsprechen denjenigen von DIN EN 1991-1-3 und DIN EN 1991-1-3/A1.

# 1 Allgemeines

## 1.1 Anwendungsbereich

### NDP zu 1.1(2)

Für Bauten in einer Höhenlage von mehr als 1 500 m müssen in jedem Einzelfall von der zuständigen Behörde entsprechende Rechenwerte festgelegt werden.



### NDP zu 1.1(3)

Es gelten die Regelungen nach Anhang A für übliche örtliche Gegebenheiten Fall A, für außergewöhnliche Bedingungen Fall B1.

### NDP zu 1.1(4)

Anhang B ist in Deutschland nicht anzuwenden.

ANMERKUNG: Der im Titel von Anhang B verwendete Begriff „außergewöhnliche Schneeverwehungen“ bezieht sich nicht auf eine Bemessungssituation nach DIN EN 1990:2002-10, 4.1.1(2).

# 2 Klassifikation von Einwirkungen

## NDP zu 2(3)

siehe NDP zu 4.3(1)

## NDP zu 2(4)

Schneeverwehungen sind nach diesem Nationalen Anhang keine außergewöhnlichen Einwirkungen. Die untere und obere Begrenzung von Schneeverwehungen bei außergewöhnlichen Schneelastansätzen nach 4.3 wird im NDP zu 5.3.6(1) und im NDP zu 6.2(2) behandelt.

# 3 Bemessungssituation

## 3.3 Außergewöhnliche Verhältnisse

### NDP zu 3.3(1)

Wo die zuständigen Stellen örtlich außergewöhnliche Schneelasten festlegen [siehe nationale Regelung zu 4.3(1)], ist auch für die besonderen örtlichen Effekte nach DIN EN 1991-1-3:2010-12, Abschnitt 6 in Verbindung mit DIN EN 1991-1-3/A1:2015-12 die Bemessungssituation nach 3.3(1) (im Anhang A als Fall B1 bezeichnet) zugrunde zu legen.

### NDP zu 3.3(2)

siehe NDP zu 1.1(4) und NDP zu 2(4)

### NDP zu 3.3(3)

siehe NDP zu 1.1(4) und NDP zu 2(4)

# 4 Schneelast auf dem Boden

## 4.1 Charakteristische Werte

### NDP zu 4.1(1)

Die charakteristischen Werte für Schneelasten auf dem Boden werden für regionale Zonen (Schneelastzonen) ermittelt.


Die folgende Abbildung zeigt die Schneelastzonenkarte für Deutschland, die in verschiedene Zonen unterteilt ist:

### Bild NA.1: Schneelastzonenkarte

|Zone|Beschreibung|
|---|---|
|Zone 1|Niedrige Schneelast|
|Zone 2|Mittlere Schneelast|
|Zone 3|Hohe Schneelast|


Hinsichtlich der genaueren Zuordnung der Schneelastzonen und des Bereichs des „Norddeutschen Tieflands“ nach Verwaltungsgrenzen wird auf die Tabelle „Zuordnung der Schneelastzonen nach Verwaltungsgrenzen“ hingewiesen.

ANMERKUNG: Die Tabelle „Zuordnung der Schneelastzonen nach Verwaltungsgrenzen“ ist über http://www.is-argebau.de oder http://www.dibt.de abrufbar.

In den Zonen 1 bis 3 sind die charakteristischen Werte der Schneelasten auf dem Boden in Abhängigkeit von der Schneelastzone und der Geländehöhe über dem Meeresniveau nach Gleichung (NA.1) bis Gleichung (NA.3) zu berechnen.

Die charakteristischen Werte in den Zonen 1a und 2a ergeben sich jeweils durch Erhöhung der Werte aus den Zonen 1 und 2 mit dem Faktor 1,25. Die Sockelbeträge (siehe Bild NA.2) werden in gleicher Weise angehoben.

Zone 1:
\[
s_k = 0,19 + 0,91 \times \left(\frac{A + 140}{760}\right)^2 \quad (NA.1)
\]

Zone 2:
\[
s_k = 0,25 + 1,91 \times \left(\frac{A + 140}{760}\right)^2 \quad (NA.2)
\]

Zone 3:
\[
s_k = 0,31 + 2,91 \times \left(\frac{A + 140}{760}\right)^2 \quad (NA.3)
\]

Dabei ist:
- sk der charakteristische Wert der Schneelast auf dem Boden, in kN/m2;
- A die Geländehöhe über Meeresniveau, in m.





|Zone|Sockelbeträge (Mindestwerte)|
|---|---|
|Zone 1|0,65 kN/m² (bis 400 m ü. d. M.)|
|Zone 2|0,85 kN/m² (bis 285 m ü. d. M.)|
|Zone 3|1,10 kN/m² (bis 255 m ü. d. M.)|

A = Höhe über dem Meeresniveau in m

sk = Schneelast in kN/m²

### Bild NA.2 - Charakteristischer Wert der Schneelast sk auf dem Boden

Für bestimmte Lagen der Schneelastzone 3 können sich höhere Werte als nach Gleichung (NA.3) ergeben. Informationen über die Schneelast in diesen Lagen sind von den örtlichen, zuständigen Stellen einzuholen.

Beispielhaft können folgende Gebiete benannt werden:
- Oberharz
- Hochlagen des Fichtelgebirges
- Bayerischer Wald

## 4.2 Weitere repräsentative Werte

### NDP zu 4.2(1)

Es gelten die empfohlenen Werte.






## 4.3 Behandlung von außergewöhnlichen Schneelasten auf dem Boden

### NDP zu 4.3(1)

Im norddeutschen Tiefland wurden in seltenen Fällen Schneelasten bis zum mehrfachen der rechnerischen Werte gemessen. Die zuständige Behörde kann in den betroffenen Regionen die Rechenwerte festlegen, die dann zusätzlich nach DIN EN 1990 als außergewöhnliche Einwirkungen zu berücksichtigen sind. Für Cesl gilt der Wert 2,3, soweit die örtlichen Behörden keine anderen Werte festsetzen.

# 5 Schneelast auf Dächern

## 5.2 Lastanordnung

### NDP zu 5.2(2)

siehe NDP zu 1.1(4)

### NDP zu 5.2(5)

Es werden keine weitergehenden nationalen Regelungen getroffen.

### NDP zu 5.2(6)

Es werden keine weitergehenden nationalen Regelungen getroffen.

### NDP zu 5.2(7)

Es gilt C_e = 1,0.

### NDP zu 5.2(8)

Im Regelfall gilt C_t = 1,0.

## 5.3 Formbeiwerte für Dächer

### 5.3.1 Allgemeines

#### NDP zu 5.3.1(1)

siehe NDP zu 1.1(4) und NDP zu 2(4)

#### NCI zu 5.3.1(2)

Bei aufgeständerten Solarthermie- und Photovoltaikanlagen auf Dächern bis 10° Neigung dürfen die Formbeiwerte vereinfacht für das Dach nach Bild NA.3 und Gleichung (NA.4) angesetzt werden. Für Anlagenhöhen h ≤ 0,5 m gilt:

\[
\mu_5 = \min \left( \frac{\gamma \cdot s_h}{k}; \text{ jedoch nicht weniger als } \mu_1 \text{ bzw. } \mu_2 \right)
\]

Dabei ist
- γ die Wichte des Schnees, die für diese Berechnung zu 2 kN/m³ angenommen werden kann.


Im Falle außergewöhnlicher Einwirkungen nach 3.3(1) (norddeutsches Tiefland) gilt für Gleichung (NA.4)
\[
\mu_5 = \min \left( \frac{\gamma}{s_{Ad}} \times h; \text{ jedoch nicht weniger als } \mu_1 \text{ bzw. } \mu_2 \right)
\]

Bei Anlagenhöhen mit \( h > 0,5 \, \text{m} \) ist zur Berücksichtigung der Verwehung \( \mu_5 \) um 10 % zu erhöhen.

Die Verwehungslänge ist nach Bild NA.3 zu bestimmen und beträgt:
\[
l_s = l_1 + 2 \times h
\]

Dabei ist für \( l_1 \) jeweils die Abmessung der Belegungsfläche in Länge und Breite zu berücksichtigen.

##### Bild NA.3

Bild NA.3 zeigt die Formbeiwerte und Verwehungslängen für Solarthermie- und Photovoltaikanlagen.

#### NDP zu 5.3.1(3)

Anstelle DIN EN 1991-1-3/A1:2015-12, Tabelle 5.2 ist folgende Tabelle NA.1 anzuwenden.


Tabelle NA.1 — Formbeiwerte für Schneelasten
|Neigungswinkel α der Dachfläche|0° ≤ α ≤ 30°|30° < α < 60°|α ≥ 60°|
|---|---|---|---|
|μ1(α)|μ1(0°) = 0,8|μ1(0°) (60° − α) / 30°|0,0|
|μ2(α)|0,8|0,8 (60° − α) / 30°|0,0|
|μ3(α)|0,8 + 0,8α / 30°|1,6|1,6|

Bei Dächern mit Neigungen ≤ 30°, deren kleinste Grundrissabmessung mehr als 50 m beträgt, ist der Formbeiwert \( \mu_1(α) \) oder \( \mu_2(α) \) nach Gleichung (NA.5) zu bestimmen:

\[
\mu_1(α) = \mu_2(α) = 0,80 + 0,20 \times \frac{(B - 50)}{200} \quad \text{für } B \leq 1,0
\]

Dabei ist \( B \) die geringere der beiden Grundrissabmessungen des Daches.






### 5.3.2 Pultdächer

#### NDP zu 5.3.2(3) 
siehe NDP zu 1.1(4) und NDP zu 2(4)

### 5.3.3 Satteldächer

#### NDP zu 5.3.3(4) 
Es gelten die Regelungen nach DIN EN 1991-1-3.

### 5.3.4 Scheddächer

#### NDP zu 5.3.4(3)
Es werden keine weitergehenden nationalen Regelungen getroffen.

#### NCI zu 5.3.4(4) 
Die Formbeiwerte für gereihte Dächer sind je nach maßgebender Dachneigung der Tabelle NA.1 zu entnehmen; statt der Formbeiwerte nach DIN EN 1991-1-3:2010-12 und DIN EN 1991-1-3/A1:2015-12, Bild 5.3 sind jedoch die Formbeiwerte nach Bild NA.4 anzuwenden.

Der Formbeiwert \( \mu_3 \) (siehe Tabelle NA.1) darf begrenzt werden auf:

\[
\mu_3 \leq \frac{\gamma \times h + \mu_2}{s_k} \tag{NA.6}
\]
bei außergewöhnlichen Einwirkungen nach 3.3(1) (norddeutsches Tiefland) auf:
\[
\mu_3 \leq \frac{\gamma \times h + \mu_2}{s_{ad}} \tag{NA.7}
\]

Dabei ist \( \gamma \) die Wichte des Schnees, die für diese Berechnung zu 2 kN/m³ angenommen werden kann.

##### Bild NA.4 — Formbeiwerte für gereihte Satteldächer und Scheddächer

Das Bild zeigt die Formbeiwerte für gereihte Satteldächer und Scheddächer, die in den entsprechenden Berechnungen verwendet werden. 
Für die Innenfelder ist dabei der mittlere Neigungswinkel \( \alpha = 0,5 (\alpha_1 + \alpha_2) \) maßgebend.
Die Schneelast auf steil stehende Fensterflächen oder auf angrenzende Bauteile kann sinngemäß nach 6.4 ermittelt werden.






### 5.3.5 Tonnendächer

#### NDP zu 5.3.5(1)

Formbeiwerte für Schneelasten sind Bild NA.5 zu entnehmen.

#### NDP zu 5.3.5(3)

Für verwehten Schnee gilt die Lastverteilung nach Bild NA.5, Fall (ii).
Fall (i)
Fall (ii)
p2 = 0,8
0,5p44
Hl = 1

##### Bild NA.5 — Formbeiwerte für Schneelasten auf Tonnendächern

### 5.3.6 Höhensprünge an Dächern

#### NDP zu 5.3.6(1), Anmerkung 1

𝜇w ist für Höhensprünge h > 0,5 m zu berücksichtigen.

Im Falle von außergewöhnlichen Einwirkungen nach 4.3(1) (norddeutsches Tiefland) muss er nicht größer angesetzt werden als

\[
\mu_w = \frac{\gamma \times h}{s_{ad}} \tag{NA.8}
\]

Für die Summe 𝜇w + 𝜇s gilt

\[
0,8 \leq \mu_w + \mu_s \leq 2,4 \tag{NA.9}
\]

Bei seitlich offenen und für die Räumung zugänglichen Vordächern (b2 ≤ 3 m) braucht unabhängig von der Größe des Höhensprunges nur die ständige/vorübergehende Bemessungssituation betrachtet zu werden. Dabei gilt die Begrenzung:

\[
0,8 \leq \mu_w + \mu_s \leq 2 \tag{NA.10}
\]


Für die alpine Region nach DIN EN 1991-1-3:2010-12 und DIN EN 1991-1-3/A1:2015-12, Bild C.2, gilt für Schneelasten sk ≥ 3,0 kN/m² die obere Begrenzung:



\[
\mu_w + \mu_s \leq 6,45, \text{ mindestens jedoch } 1,2.
\]

Bei Anordnung von Schneefanggittern oder vergleichbaren Einrichtungen darf auf den Ansatz von μs verzichtet werden.

#### NDP zu 5.3.6(1), Anmerkung 2

Es gelten die empfohlenen Werte.

#### NDP zu 5.3.6(1), Anmerkung 3

Es gilt das empfohlene Verfahren.

#### NDP zu 5.3.6(3)

siehe NDP zu 1.1(4)

# 6 Örtliche Effekte

## 6.2 Verwehungen an Wänden und Aufbauten

### NDP zu 6.2(2)
siehe NDP zu 1.1(4)
Im Falle außergewöhnlicher Einwirkungen nach 4.3(1) (norddeutsches Tiefland) gilt:

\[
\mu_2 = \gamma \times h
\]

Wände und Aufbauten mit einer Ansichtsfläche unter 1 m² oder einer Höhe unter 0,5 m brauchen nicht berücksichtigt werden.

## 6.3 Schneeüberhang an Dachtraufen

### NDP zu 6.3(1)
Der Nachweis auskragender Dachteile für die Trauflast ist unabhängig von der Höhenlage des Bauortes zu führen.

### NDP zu 6.3(2)

Der Beiwert k für die Form des Überhanges darf in Deutschland mit k = 0,4 abgemindert werden. Sofern über die Dachfläche verteilt Schneefanggitter oder vergleichbare Einrichtungen angeordnet werden, die das Abgleiten von Schnee wirksam verhindern und nach Abschnitt 6.4 bemessen sind, kann auf den Ansatz der Linienlast ganz verzichtet werden.

## 6.4 Schneelasten an Schneefanggittern und Dachaufbauten

### NCI zu 6.4(1)
Werden Schneefanggitter zur Reduzierung der Schneelast auf die Tragkonstruktion, z. B. Lasten aus abgleitenden Schneemassen auf tiefer liegende Dachflächen bei Höhensprüngen (siehe 5.3.6), angeordnet oder sind Dachaufbauten vorgesehen, die abgleitende Schneemassen anstauen, so ist eine Schneelast (Fs je m Länge) nach dem folgenden Bild NA.6 anzusetzen.

#### Bild NA.6 — Schneelast auf Schneefanggitter

Das Bild zeigt die Verteilung der Schneelast auf Schneefanggitter. Es ist wichtig, die korrekte Anordnung und Dimensionierung der Schneefanggitter zu beachten, um die Sicherheit der Dachkonstruktion zu gewährleisten.

### NDP zu Anhang A (informativ)
siehe NDP zu 1.1(4) und 4.3(1)

Nach den nationalen Regelungen zu 1.1(4), 2(4) und 4.3(1) ist für außergewöhnliche Bedingungen ausschließlich Fall B1 zutreffend. Für die Fälle B2 und B3 werden keine Regelungen getroffen.




# NCI Anhang NA.F (informativ)

## Eislasten

### NA.F.1 Allgemeines

Die Vereisung (Eisregen oder Raueis) hängt von den meteorologischen Einflüssen wie Lufttemperatur,
relative und absolute Luftfeuchtigkeit und Wind ab, die mit der Geländeform und der Geländehöhe über NN
stark wechseln.

Wegen der vielfältigen Einflussfaktoren können zur Art und Stärke des Eisansatzes allgemeine Angaben nur
bis zu Höhenlagen ≤ 600 m ü. NN und bis zu Bauwerkshöhen von 50 m über Gelände gemacht werden.
Anhaltswerte zur Ermittlung der Lasten für das Gebiet der Bundesrepublik Deutschland werden für Lagen
bis zu 600 m ü NN im informativen Anhang NA.F gegeben. In allen anderen Fällen und für besonders
exponierte Lagen ist bereits in der Planung in Abstimmung mit der zuständigen Behörde festzulegen,
welcher Eisansatz zu berücksichtigen ist.

Bei filigranen Bauteilen kann für die Bemessung ein Eislastansatz anstelle des Schneelastansatzes
maßgebend werden. Neben dem erhöhten Gewicht sollte dabei auch die größere Windangriffsfläche
beachtet werden.

### NA.F.2 Vereisungsklassen

#### NA.F.2.1 Allgemeines

Die Art des Eisansatzes hängt von den meteorologischen Bedingungen ab, die während des
Vereisungsvorganges am Bauort herrschen. Für die Berechnung dürfen zwei typische Fälle entsprechend
NA.F.2.2 und NA.F.2.3 klassifiziert werden.

#### NA.F.2.2 Vereisungsklassen G

Es wird eine allseitige Ummantelung der Bauteile mit Klareis (gefrierende Nebellagen) oder Glatteis
(gefrierender Regen) angenommen, die durch die Dicke der Eisschicht in Zentimeter charakterisiert ist
(siehe Bild NA.F.1). So bedeutet z. B. die Vereisungsklasse G 1 einen allseitigen Eisansatz von t = 1 cm und
entsprechend für G 2 mit t = 2 cm.

Für das Gebiet der Bundesrepublik Deutschland dürfen die Vereisungsklassen G 1 oder G 2 als maßgebend
angenommen werden.

Die Eisrohwichte für Klareis und Glatteis darf mit 9 kN/m3 angesetzt werden.


##### Bild NA.F.1 — Allseitiger Eismantel

Das Bild zeigt eine schematische Darstellung eines allseitigen Eismantels, der sich um ein Bauteil bildet.

#### NA.F.2.3 Vereisungsklassen R

Die vorherrschende Windrichtung während der Vereisung des Bauwerks führt zum Aufbau einer einseitigen, gegen den Wind anwachsenden kompakten Eisfahne. Sie ist in Tabelle NA.F.1 durch das Gewicht des an einem dünnen Stab angelagerten Eises definiert. Dies gilt für Stäbe beliebiger Querschnittsform bis zu einer Profilbreite von 300 mm.

Tabelle NA.F.1 — Vereisungsklassen für Raueis
|Vereisungsklasse|Eisgewicht an einem Stab (∅ ≤ 300 mm) kN/m|
|---|---|
|R 1|0,005|
|R 2|0,009|
|R 3|0,016|
|R 4|0,028|
|R 5|0,050|

Im Flachland und bis in die unteren Lagen der Mittelgebirge der Bundesrepublik Deutschland dürfen die Vereisungsklassen R 1 bis R 3 angenommen werden. Analog zur Windgeschwindigkeit gilt das in Tabelle NA.F.1 angegebene Eisgewicht in 10 m Höhe über Gelände. Im Falle abweichender Bauteilhöhen ist der Höhenfaktor kZ nach NA.F.3.2 zu berücksichtigen.

Die Eisrohwichte für Raueis darf mit 5 kN/m3 angesetzt werden. Die schematisierten Formen einer anwachsenden kompakten Eisfahne sind für nicht verdrehbare Stabquerschnitte in Bild NA.F.2 dargestellt. Bei verdrehbaren Querschnitten (z. B. Seilen) kann es durch die Rotation zu einer allseitigen Eisanlagerung (Eiswalze) kommen. Die Schichtdicke darf aus den Eisgewichten nach Tabelle NA.F.1 berechnet werden.

Mit wachsender Querschnittsbreite nimmt die Länge der Eisfahne ab, jedoch nur bis zu einer Breite von 300 mm. Für breitere Querschnitte darf der Wert für 300 mm angenommen werden, sodass sich für diese Bauteile höhere Eisgewichte je Längeneinheit ergeben. Weitere Angaben dazu sind in [1] zu finden.

Für Fachwerke ergibt sich die Eislast als Summe der Eislasten der Einzelstäbe, wobei geometrische Überschneidungen abgezogen werden dürfen.


|Typ A|Typ B|
|---|---|
|8t|4w|

|Typ C|Typ D|
|---|---|
|8t|5w|

|Typ E|Typ F|
|---|---|
|5w|2.5w|

Legende
Phase 1; Hierbei tritt noch kein Breitenwachstum (t) ein
Phase 2; Hierbei tritt nach Abschluss der Phase 1 Breitenwachstum (t) ein

Begriffsdefinitionen
- W: die Breite des Stabquerschnitts ohne Vereisung in mm;
- D: die Gesamtbreite des vereisten Stabes in mm;
- L: die Länge der Eisfahne in windwärtiger Richtung in mm;
- t: die Breite des Eisablagerungsansatzes in mm.

##### Bild NA.F.2 — Eisfahnen von Stäben mit unterschiedlicher Querschnittsform


Die Maße der Eisfahnen für die in Bild NA.F.2 dargestellten Stabtypen dürfen der Tabelle NA.F.2 und Tabelle NA.F.3 entnommen werden (sinngemäß nach [1]).

|Stabquerschnitt Typ A, B, C und D|Stabbreite W (mm)|10|30|100|300| | | | |
|---|---|---|---|---|---|---|---|---|---|
|Eisklasse R 1 (kN/m)|0,005|56|23|36|35|13|100|4|300|
|Eisklasse R 2 (kN/m)|0,009|80|29|57|40|23|100|8|300|
|Eisklasse R 3 (kN/m)|0,016|111|37|86|48|41|100|14|300|

|Stabquerschnitt Typ E und F|Stabbreite W (mm)|10|30|100|300| | | | |
|---|---|---|---|---|---|---|---|---|---|
|Eisklasse R 1 (kN/m)|0,005|55|22|29|34|0|100|0|300|
|Eisklasse R 2 (kN/m)|0,009|79|28|51|39|0|100|0|300|
|Eisklasse R 3 (kN/m)|0,016|111|36|81|47|9|100|0|300|

### NA.F.3 Vereisungsklassen in Deutschland

#### NA.F.3.1 Bauteile auf Geländehöhe

Aufgrund der meteorologischen und topographischen Verhältnisse wird Deutschland nach Bild NA.F.3 in die folgenden Eiszonen unterteilt (siehe [2]).







##### Bild NA.F.3 — Eiszonenkarte Bundesrepublik Deutschland

Für die dargestellten Zonen sollten die Vereisungsklassen entsprechend Tabelle NA.F.4 alternativ untersucht werden.

|Zonen|Vereisungsklassen|
|---|---|
|Z = 2|2 = 2|
|Z = 3|2 = 3|
|Z = 4|2 = 4|







Tabelle NA.F.4 — Vereisungsklassen im Gebiet der Bundesrepublik Deutschland
**Vereisungsklassen in Deutschland**
|Zone|Region|Vereisungsklasse|
|---|---|---|
|1|Küste|G 1, R 1|
|2|Binnenland|G 2, R 1|
|3|Mittelgebirge A ≤ 400 m|R 2|
|4|Mittelgebirge 400 m < A ≤ 600 m|R 3|

Die Vereisungsklassen decken normale Verhältnisse ab. In besonders exponierten oder gut abgeschirmten Lagen darf die maßgebende Vereisungsklasse zutreffender durch ein meteorologisches Gutachten festgelegt werden. Für Höhenlagen A oberhalb 600 m ü. NN sollte die Vereisungsklasse durch ein Gutachten in Abstimmung mit der zuständigen Behörde festgelegt werden.

#### NA.F.3.2 Eisansatz in größeren Höhen über Gelände

Für R-Klassen gilt, dass bedingt durch die anwachsende Windgeschwindigkeit der Eisansatz mit der Höhe über Gelände zunimmt. Für Bauteile bis h = 50 m über Gelände wird die Menge des Eisansatzes mit dem Höhenfaktor

\[
k_Z = 1 + \frac{h - 10}{100} \tag{NA.F.1}
\]

vergrößert (siehe Bild NA.F.4). Die Höhe h ist in Meter einzusetzen.

Legende
- h: Höhe über Gelände in m
- kZ: Höhenfaktor

##### Bild NA.F.4 — Höhenfaktor kZ

Das Bild zeigt den Zusammenhang zwischen der Höhe über Gelände und dem Höhenfaktor kZ. Es ist zu beachten, dass der Eisansatz für G-Klassen für Bauteile mit Klareis bis zu 50 m über Gelände als gleich bleibend angesetzt werden darf.

#### NA.F.3.3 Windlast auf vereiste Baukörper

Die Windlast auf vereiste Baukörper wird nach DIN EN 1991-1-4 bestimmt.
Durch Eisansatz ändert sich die Querschnittsform der Bauteile und damit der Windkraftbeiwert und die Bezugsfläche, bei Fachwerken auch der Völligkeitsgrad. Dies ist in der Berechnung zu berücksichtigen.

In den Vereisungsklassen G sollte mit den allseitig geometrisch vergrößerten Querschnitten gerechnet werden. Ausgehend von den Windkraftbeiwerten \( c_{f0} \) ohne Eisansatz können im Bild NA.F.5 die veränderten Werte \( c_{fi} \) für Eisansatz abgelesen oder linear interpoliert werden. Die Windkraftbeiwerte tendieren mit zunehmender Vereisung auf einen einheitlichen Wert hin.

|G|cfi|a|
|---|---|---|
|G1|1,5|eisfrei|
|G2|1,4|eisfrei|
|G3|0,5|eisfrei|

##### Bild NA.F.5 — Veränderte Windkraftbeiwerte \( c_{fi} \) bei allseitigem Eisansatz

Bei den Raueisklassen R sollte ungünstig davon ausgegangen werden, dass der Wind quer zu den Eisfahnen bläst. Ausgehend von den Windkraftbeiwerten \( c_{f0} \) ohne Eisansatz können in Bild NA.F.6 die veränderten Werte \( c_{fi} \) für Eisansatz abgelesen oder linear interpoliert werden.

|R|cfi|a|
|---|---|---|
|R1|1,5|eisfrei|
|R2|1,6|eisfrei|
|R3|0,5|eisfrei|

##### Bild NA.F.6 — Veränderte Windkraftbeiwerte \( c_{fi} \) bei Raueis

Für dünne und für stabförmige Bauglieder bis zur Breite von 300 mm können die vergrößerten Windangriffsflächen der Tabelle NA.F.2 und Tabelle NA.F.3 entnommen werden.

Für Bauteile mit einer Breite über 300 mm lassen sich die durch Eisansatz veränderten Windkraftbeiwerte nach [1] abschätzen.
"""




In [13]:
import uuid
import re
from typing import List, Dict, Any

class Neo4jNode:
    def __init__(self, label: str, properties: Dict[str, Any]):
        self.label = label
        self.properties = properties
        self.id = str(uuid.uuid4())

    def to_cypher(self) -> str:
        props = ', '.join([f"{k}: {repr(v)}" for k, v in self.properties.items()])
        return f"CREATE (:{self.label} {{id: '{self.id}', {props}}})"

class Neo4jRelationship:
    def __init__(self, start_node: Neo4jNode, end_node: Neo4jNode, rel_type: str):
        self.start_node = start_node
        self.end_node = end_node
        self.rel_type = rel_type

    def to_cypher(self) -> str:
        return f"MATCH (a), (b) WHERE a.id = '{self.start_node.id}' AND b.id = '{self.end_node.id}' CREATE (a)-[:{self.rel_type}]->(b)"

def extract_num_title(name: str) -> tuple:
    match = re.match(r'^(\d+(?:\.\d+)*)\s*(.*)$', name)
    if match:
        return match.group(1), match.group(2)
    return '', name

def process_section(section: Section, parent: Neo4jNode = None, sequence_num: int = 1) -> List[str]:
    queries = []
    num, title = extract_num_title(section.name)
    
    if section.level == 1:
        node = Neo4jNode('Chapter', {'num': num, 'title': title})
    else:
        node = Neo4jNode('Section', {'num': num, 'title': title, 'sequence_num': sequence_num})
    
    queries.append(node.to_cypher())
    
    if parent:
        rel = Neo4jRelationship(node, parent, 'PART_OF')
        queries.append(rel.to_cypher())
    
    for i, content in enumerate(section.content, start=1):
        if content.strip():
            chunk = Neo4jNode('Chunk', {'sequence_num': i, 'content': content})
            queries.append(chunk.to_cypher())
            chunk_rel = Neo4jRelationship(chunk, node, 'PART_OF')
            queries.append(chunk_rel.to_cypher())
    
    for i, subsection in enumerate(section.subsections, start=1):
        queries.extend(process_section(subsection, node, i))
    
    return queries

def generate_neo4j_queries(root: Section) -> List[str]:
    return process_section(root)


In [14]:
# executing the markdown parsing process and printing each generated cypher query creating the nodes and relationships
root = parse_markdown(markdown_text)
queries = generate_neo4j_queries(root)

for query in queries:
    print(query)
    print("---------")

#root = parse_markdown(markdown_text)
#print_structure(root)

CREATE (:Section {id: '30019937-0ac8-451f-9d78-eff503026400', num: '', title: 'Root', sequence_num: 1})
---------
CREATE (:Chapter {id: '45c74103-0a0e-4925-960e-037b520cc190', num: '', title: 'NA.1 Anwendungsbereich'})
---------
MATCH (a), (b) WHERE a.id = '45c74103-0a0e-4925-960e-037b520cc190' AND b.id = '30019937-0ac8-451f-9d78-eff503026400' CREATE (a)-[:PART_OF]->(b)
---------
CREATE (:Chunk {id: 'd101c23c-7571-4967-b0f5-0c060b21e9cd', sequence_num: 1, content: 'Dieser Nationale Anhang enthält nationale Festlegungen für die Grundsätze zur Bestimmung der Werte von Schneelasten für die Berechnung und Bemessung von Hoch- und Ingenieurbauten, die bei der Anwendung von DIN EN 1991-1-3 und DIN EN 1991-1-3/A1 in Deutschland zu berücksichtigen sind.'})
---------
MATCH (a), (b) WHERE a.id = 'd101c23c-7571-4967-b0f5-0c060b21e9cd' AND b.id = '45c74103-0a0e-4925-960e-037b520cc190' CREATE (a)-[:PART_OF]->(b)
---------
CREATE (:Chunk {id: '615da39c-39b7-4932-ac4b-c7a8c4093106', sequence_num: 3, c

In [15]:
print_structure(root)

Root (Level 0)
  NA.1 Anwendungsbereich (Level 1)
    Content: Dieser Nationale Anhang enthält nationale Festlegungen für die Grundsätze zur Bestimmung der Werte von Schneelasten für die Berechnung und Bemessung von Hoch- und Ingenieurbauten, die bei der Anwendung von DIN EN 1991-1-3 und DIN EN 1991-1-3/A1 in Deutschland zu berücksichtigen sind.
    Content: Dieser Nationale Anhang gilt nur in Verbindung mit DIN EN 1991-1-3 und DIN EN 1991-1-3/A1.
  NA.2 Nationale Festlegungen zur Anwendung von DIN EN 1991-1-3 und DIN EN 1991-1-3/A1 (Level 1)
    NA 2.1 Allgemeines (Level 2)
      Content: DIN EN 1991-1-3 weist an den folgenden Textstellen die Möglichkeit nationaler Festlegungen (NDP) aus:
      Content: - 1.1(2), 1.1.(3), 1.1.(4) - 2(3), 2(4) - 3.3(1), 3.3(2), 3.3(3) - 4.1(1), 4.1(2), 4.2(1), 4.3(1) - 5.2(2), 5.2(5), 5.2(6), 5.2(7), 5.2(8), 5.3.1(1) Anmerkung zu Tabelle 5.2, 5.3.2(3), 5.3.3(4), 5.3.4(3), 5.3.4(4), 5.3.5(1), 5.3.5(3), 5.3.6(1), 5.3.6(3) - 6.2(2), 6.3(1), 6.3(2) - A(1) 

In [17]:
# Define your Neo4j connection details
neo4j_uri = ""  # Change to your Neo4j URI
username = ""  # Change to your username
password = ""  # Change to your password
from neo4j import GraphDatabase
driver = GraphDatabase.driver(neo4j_uri, auth=(username, password))

In [16]:
# execute database queries
with driver.session() as session:
    for query in queries:
        session.run(query)
    
driver.close()

/var/folders/qj/lhzh7m7s69n_ghf6fn2lyq1w0000gn/T/ipykernel_27555/1728670869.py:1: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


In [26]:
# function for replacing a node with another node by their IDs
# necessary for replacing the blank root node with the actual root node representing the document itself
def replace_node(driver, node_id_to_replace, replacement_node_id):
    with driver.session() as session:
        # Transfer outgoing relationships from node_id_to_replace to replacement_node_id
        transfer_outgoing_query = """
        MATCH (n1)-[r]->(n2), (n_new) WHERE n1.id = $node_id_to_replace AND n_new.id = $replacement_node_id
        CALL apoc.create.relationship(n_new, type(r), properties(r), n2) YIELD rel
        DELETE r
        RETURN n1
        """

        # Transfer incoming relationships to node_id_to_replace to replacement_node_id
        transfer_incoming_query = """
        MATCH (n2)-[r]->(n1), (n_new) WHERE n1.id = $node_id_to_replace AND n_new.id = $replacement_node_id
        CALL apoc.create.relationship(n2, type(r), properties(r), n_new) YIELD rel
        DELETE r
        RETURN n1
        """

        # Execute the queries to transfer relationships
        session.run(transfer_outgoing_query, node_id_to_replace=node_id_to_replace, replacement_node_id=replacement_node_id)
        session.run(transfer_incoming_query, node_id_to_replace=node_id_to_replace, replacement_node_id=replacement_node_id)

        # Delete the original node
        delete_query = """
        MATCH (n) WHERE n.id = $node_id_to_replace
        DELETE n
        """
        session.run(delete_query, node_id_to_replace=node_id_to_replace)


# Example usage:
# Replace node with ID 123 with node with ID 456
# You need to initialize a driver first, like:
# driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "password"))
# replace_node(driver, 123, 456)

In [28]:
# executing the node replacement
replace_node(driver, '30019937-0ac8-451f-9d78-eff503026400', '14bab7c1-3ef5-462d-b37f-55637f530abf')